In [1]:
import requests
import pandas as pd
import json
import plotly.express as px

In [2]:
# lits of top french cities
list_cities = ["Mont Saint Michel", "St Malo", "Bayeux", "Le Havre", "Rouen", "Paris", "Amiens", "Lille", "Strasbourg", 
"Chateau du Haut Koenigsbourg", "Colmar", "Eguisheim", "Besancon", "Dijon", "Annecy", "Grenoble", "Lyon", "Gorges du Verdon", "Bormes les Mimosas",
"Cassis", "Marseille", "Aix en Provence", "Avignon", "Uzes", "Nimes", "Aigues Mortes", "Saintes Maries de la mer", "Collioure", "Carcassonne",
"Ariege", "Toulouse", "Montauban", "Biarritz", "Bayonne", "La Rochelle"]


In [3]:
# create a dataframe
df_cities = pd.DataFrame(list_cities)
df_cities['id_city'] = range(0, len(df_cities))
df_cities = df_cities.rename(columns={0: 'cities'})
df_cities = df_cities[['id_city', 'cities']]

In [4]:
# check the dataframe
df_cities.head()

,id_city,cities
0,0,Mont Saint Michel
1,1,St Malo
2,2,Bayeux
3,3,Le Havre
4,4,Rouen


In [5]:
#import on df 'lat' and 'lon' from nomitam.openstreetmap 
url_cities='https://nominatim.openstreetmap.org/search.php?q={}&format=jsonv2'

latitude=[]
longitude=[]

for i in range(len(list_cities)):
    r=requests.get(url_cities.format(list_cities[i]))
    latitude.append(r.json()[0]['lat'])
    longitude.append(r.json()[0]['lon'])

df_cities['latitude']=latitude
df_cities['longitude']=longitude 
display(df_cities.head())
df_cities.shape

,id_city,cities,latitude,longitude
0,0,Mont Saint Michel,48.6359541,-1.511459954959514
1,1,St Malo,48.649518,-2.0260409
2,2,Bayeux,49.2764624,-0.7024738
3,3,Le Havre,49.4938975,0.1079732
4,4,Rouen,49.4404591,1.0939658


(35, 4)

In [6]:
# change type as float for latitude and longitude (use for ploting after)
df_cities['latitude'] = df_cities['latitude'].astype('float')
df_cities['longitude'] = df_cities['longitude'].astype('float')

In [8]:
#import weather, rain and temp from open-meteo
url_weather = 'https://api.open-meteo.com/v1/forecast?latitude={}&longitude={}&daily=weathercode,apparent_temperature_max,apparent_temperature_min,rain_sum&timeformat=unixtime&timezone=Europe%2FLondon&start_date={}&end_date={}'

start_date = '2023-01-31'
end_date = '2023-02-06'
temp_min=[]
temp_max=[]
rain=[]
weather= []

for i in range(len(list_cities)):
    rw=requests.get(url_weather.format(df_cities['latitude'][i], df_cities['longitude'][i], start_date, end_date))
    temp_min.append(rw.json()['daily']['apparent_temperature_min'])
    temp_max.append(rw.json()['daily']['apparent_temperature_max'])
    rain.append(rw.json()['daily']['rain_sum'])
    weather.append(rw.json()['daily']['weathercode'])

In [9]:
# create dataframe for the news data
temp_min = pd.DataFrame(temp_min)
temp_max =pd.DataFrame(temp_max)
rain = pd.DataFrame(rain)
weather = pd.DataFrame(weather)

In [10]:
#i choose to take apparent temperature and i make the means with min and max
temp_min['moy'] = round(temp_min.mean(axis=1), 1)
temp_max['moy'] = round(temp_max.mean(axis=1), 1)
temp = round((temp_min['moy'] + temp_max['moy'])/2, 1)
temp.head()

0    5.0
1    5.1
2    4.8
3    5.5
4    4.2
Name: moy, dtype: float64

In [11]:
# i do the same for the rain
rain['moy'] = round(rain.mean(axis=1), 1)
rain['moy'].head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: moy, dtype: float64

In [12]:
# i translate the code of weather into description to be  more usefull and i apply the mode

#Code 	Description
#0 	Clear sky
#1, 2, 3 	Mainly clear, partly cloudy, and overcast
#45, 48 	Fog and depositing rime fog
#51, 53, 55 	Drizzle: Light, moderate, and dense intensity
#56, 57 	Freezing Drizzle: Light and dense intensity
#61, 63, 65 	Rain: Slight, moderate and heavy intensity
#66, 67 	Freezing Rain: Light and heavy intensity
#71, 73, 75 	Snow fall: Slight, moderate, and heavy intensity
#77 	Snow grains
#80, 81, 82 	Rain showers: Slight, moderate, and violent
#85, 86 	Snow showers slight and heavy
#95 * 	Thunderstorm: Slight or moderate
#96, 99 * 	Thunderstorm with slight and heavy hail
weather.replace({0:'Clear sky', 1:'Mainly clear', 2:'partly cloudy', 3: 'Cloudly', 45:'Fog', 48:'Fog', 51:'Drizzle', 53:'Drizzle',
                  61:'Rain', 71:'Snow', 73:'Snow', 80:'Rain showers', 85:'Snow showers'}, inplace=True)
weather['mode'] = weather.mode(axis=1)[0]
weather.head(5)

,0,1,2,3,4,5,6,mode
0,Fog,Cloudly,Cloudly,Fog,Fog,Fog,Fog,Fog
1,Cloudly,Cloudly,Cloudly,partly cloudy,Fog,Fog,Fog,Cloudly
2,Cloudly,Cloudly,Cloudly,Cloudly,partly cloudy,Fog,Fog,Cloudly
3,Cloudly,Cloudly,Cloudly,Cloudly,Cloudly,Cloudly,Fog,Cloudly
4,Fog,Cloudly,Cloudly,Cloudly,Cloudly,Fog,Fog,Cloudly


In [13]:
#adding the news columns to th dataframe
df_cities['temp'] = temp
df_cities['rain'] = rain['moy']
df_cities['weather'] = weather['mode']
df_cities.head(35)

,id_city,cities,latitude,longitude,temp,rain,weather
0,0,Mont Saint Michel,48.635954,-1.511460,5.0,0.0,Fog
1,1,St Malo,48.649518,-2.026041,5.1,0.0,Cloudly
2,2,Bayeux,49.276462,-0.702474,4.8,0.0,Cloudly
3,3,Le Havre,49.493898,0.107973,5.5,0.0,Cloudly
4,4,Rouen,49.440459,1.093966,4.2,0.0,Cloudly
5,5,Paris,48.858890,2.320041,4.0,0.0,Cloudly
6,6,Amiens,49.894171,2.295695,3.7,0.0,Cloudly
7,7,Lille,50.636565,3.063528,3.4,0.0,Cloudly
8,8,Strasbourg,48.584614,7.750713,2.4,0.1,Rain
9,9,Chateau du Haut Koenigsbourg,48.249490,7.344296,-1.1,0.8,Cloudly


In [14]:
# save file to csv
df_cities.to_csv('src/cities_weather.csv', index=False)

In [3]:
# Order the data, with less rain and higher temperature
df_cities = df_cities.sort_values(by=['rain', 'temp'], ascending=[True, False]).reset_index(drop=True)

In [4]:
df_cities.head()

,id_city,cities,latitude,longitude,temp,rain,weather
0,18,Bormes les Mimosas,43.150697,6.341928,6.3,0.0,Cloudly
1,19,Cassis,43.214036,5.539632,6.0,0.0,Cloudly
2,3,Le Havre,49.493898,0.107973,5.5,0.0,Cloudly
3,25,Aigues Mortes,43.565823,4.191284,5.4,0.0,Clear sky
4,27,Collioure,42.525050,3.083155,5.4,0.0,Cloudly


In [17]:
#ploting the best 5 cities on a map
fig = px.scatter_mapbox(df_cities.iloc[:5,:], lat="latitude", lon="longitude",size='temp',hover_name='cities',
                         zoom=4.5, title= 'top 5 french cities by temperature')
fig.update_layout(mapbox_style="open-street-map",width = 650, height = 650)
fig.show()